#  Fill null values using KNN

In this Notebook , I'm going to consider column " Outlet_Size " and show you how we can fill the null values for the selcted column using KNN , you can apply the same for other columns 



In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline



In [2]:
# This param grid is for KNN

param_grid=[{'weights':['uniform','distance'],'n_neighbors':[3,4,5,6,7,8,9,10]}]


In [3]:
param_grid

[{'weights': ['uniform', 'distance'],
  'n_neighbors': [3, 4, 5, 6, 7, 8, 9, 10]}]

In [20]:
# Here I'm considering BIG Mart sales data 

data=pd.read_csv("https://raw.githubusercontent.com/karndeepsingh/MACHINE-LEARNING-PROJECTS/master/BIG%20MART%20ITEM%20SALE%20DATASET/Train.csv")

In [21]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [22]:
# As i said earlier, we are going to consider Outlet Size to demonstrate how we can fill NA values 

data.isnull().sum() 

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [23]:
# Lets check whether item identifer can help us on finding the outlet size by checking the value counts
data['Item_Identifier'].value_counts()

FDW13    10
FDG33    10
NCB18     9
FDW49     9
FDQ40     9
DRE49     9
NCJ30     9
FDV60     9
FDV38     9
FDX04     9
FDW26     9
NCF42     9
FDT07     9
NCI54     9
FDD38     9
FDF52     9
NCL31     9
FDX20     9
DRN47     9
FDO19     9
NCQ06     9
FDG09     9
FDX31     9
FDU12     9
FDP25     9
NCY18     9
FDF56     9
DRP35     8
FDL20     8
FDZ26     8
         ..
FDE39     2
DRC24     2
FDZ50     2
FDD22     2
FDD48     2
FDH22     2
FDU43     2
FDE38     2
FDA48     2
FDR03     2
DRI59     2
FDI36     2
FDM38     2
FDU09     2
FDF38     2
FDG28     2
FDB10     2
DRG25     2
NCM42     2
FDT33     2
NCS41     2
FDQ60     1
FDN52     1
FDE52     1
FDY43     1
DRF48     1
FDO33     1
FDC23     1
FDT35     1
FDK57     1
Name: Item_Identifier, Length: 1559, dtype: int64

In [24]:
# Clearly we can see that it wont help us in finding us in outlook size , we can aramse delete it ( one less to consider :)

In [25]:
data.drop('Item_Identifier',axis=1,inplace=True)

In [26]:
# Fill the null values with Others , it will help us in analysis 
data['Outlet_Size'].fillna("Others",inplace=True)

In [27]:
# Let's group with outlet identifier and see whether each identifier consists of specific outlet size 

# Here i have grouped with out identifier and out size 

data.groupby(["Outlet_Identifier","Outlet_Size"])["Outlet_Size"].count()

Outlet_Identifier  Outlet_Size
OUT010             Others         555
OUT013             High           932
OUT017             Others         926
OUT018             Medium         928
OUT019             Small          528
OUT027             Medium         935
OUT035             Small          930
OUT045             Others         929
OUT046             Small          930
OUT049             Medium         930
Name: Outlet_Size, dtype: int64

From the above results , we can see that each outlet identifier has only one outlet size type , we will place one category for each outlet identifier

In [28]:
# One way we can figure out outlet size is based on the outlet sale , let's see how it goes

In [29]:
data.groupby(["Outlet_Identifier","Outlet_Size"])["Item_Outlet_Sales"].mean()


Outlet_Identifier  Outlet_Size
OUT010             Others          339.351662
OUT013             High           2298.995256
OUT017             Others         2340.675263
OUT018             Medium         1995.498739
OUT019             Small           340.329723
OUT027             Medium         3694.038558
OUT035             Small          2438.841866
OUT045             Others         2192.384798
OUT046             Small          2277.844267
OUT049             Medium         2348.354635
Name: Item_Outlet_Sales, dtype: float64

In [30]:
# Didn't expect this :D , some  medium outlets have more income than higher outlets , so this option is dropped 

In [31]:
abc=pd.pivot_table(data,index=["Outlet_Identifier","Outlet_Size"],columns=["Outlet_Type"],aggfunc=np.mean)
abc

Item_MRP                    \
Outlet_Type                   Grocery Store Supermarket Type1   
Outlet_Identifier Outlet_Size                                   
OUT010            Others         140.777594               NaN   
OUT013            High                  NaN        141.425982   
OUT017            Others                NaN        139.421119   
OUT018            Medium                NaN               NaN   
OUT019            Small          139.787088               NaN   
OUT027            Medium                NaN               NaN   
OUT035            Small                 NaN        143.122481   
OUT045            Others                NaN        140.950246   
OUT046            Small                 NaN        142.057387   
OUT049            Medium                NaN        140.297699   

                                                                   \
Outlet_Type                   Supermarket Type2 Supermarket Type3   
Outlet_Identifier Outlet_Size                                       
OUT010            Others                    NaN               NaN   
OUT013            High                      NaN               NaN   
OUT017            Others                    NaN               NaN   
OUT018            Medium             141.678634               NaN   
OUT019            Small                     NaN               NaN   
OUT027            Medium                    NaN        139.801791   
OUT035            Small                     NaN               NaN   
OUT045            Others                    NaN               NaN   
OUT046            Small                     NaN               NaN   
OUT049            Medium                    NaN               NaN   

                              Item_Outlet_Sales                    \
Outlet_Type                       Grocery Store Supermarket Type1   
Outlet_Identifier Outlet_Size                                       
OUT010            Others             339.351662               NaN   
OUT013            High                      NaN       2298.995256   
OUT017            Others                    NaN       2340.675263   
OUT018            Medium                    NaN               NaN   
OUT019            Small              340.329723               NaN   
OUT027            Medium                    NaN               NaN   
OUT035            Small                     NaN       2438.841866   
OUT045            Others                    NaN       2192.384798   
OUT046            Small                     NaN       2277.844267   
OUT049            Medium                    NaN       2348.354635   

                                                                   \
Outlet_Type                   Supermarket Type2 Supermarket Type3   
Outlet_Identifier Outlet_Size                                       
OUT010            Others                    NaN               NaN   
OUT013            High                      NaN               NaN   
OUT017            Others                    NaN               NaN   
OUT018            Medium            1995.498739               NaN   
OUT019            Small                     NaN               NaN   
OUT027            Medium                    NaN       3694.038558   
OUT035            Small                     NaN               NaN   
OUT045            Others                    NaN               NaN   
OUT046            Small                     NaN               NaN   
OUT049            Medium                    NaN               NaN   

                              Item_Visibility                    \
Outlet_Type                     Grocery Store Supermarket Type1   
Outlet_Identifier Outlet_Size                                     
OUT010            Others             0.101457               NaN   
OUT013            High                    NaN          0.059957   
OUT017            Others                  NaN          0.061377   
OUT018            Medium                  NaN               NaN   
OUT019            Small   

In [32]:
# Let's try out with Outlet Type 

In [33]:
#data.groupby(["Outlet_Identifier","Outlet_Size","Outlet_Type"])["Outlet_Type"].count()

data.groupby(["Outlet_Identifier","Outlet_Type","Outlet_Size"])["Outlet_Size"].count()


Outlet_Identifier  Outlet_Type        Outlet_Size
OUT010             Grocery Store      Others         555
OUT013             Supermarket Type1  High           932
OUT017             Supermarket Type1  Others         926
OUT018             Supermarket Type2  Medium         928
OUT019             Grocery Store      Small          528
OUT027             Supermarket Type3  Medium         935
OUT035             Supermarket Type1  Small          930
OUT045             Supermarket Type1  Others         929
OUT046             Supermarket Type1  Small          930
OUT049             Supermarket Type1  Medium         930
Name: Outlet_Size, dtype: int64

In [34]:
# Still No luck , we can't split the size of outlet based on type

# Now we predict the Null values by training a model on available data 

Now to justify the heading of this notebook we should use KNN to fill missing values , surely we can use Decision Trees to do the same task but we dont have enough data to train ( they are greedy ) and other way is using unsupervised algorithems ( like DBSCN , GMM ) we will skip this option for now but later we will do it :) 

In [35]:
# lets drop out all the null values as we need to train the model ( train is not a correct word for KNN )
print(data.shape)
data.dropna(inplace=True)
data.shape

(8523, 11)


(7060, 11)

In [36]:
# Convert all the categorical data into numerical , there are lot of ways to do this but here we are using One Hot Encoding 


# Just before doing that , we got infomation that our Item_Fat_Content has some group names which have same meaning but they
# are in differet format

data['Item_Fat_Content'].value_counts()

Low Fat    4222
Regular    2388
LF          260
reg         106
low fat      84
Name: Item_Fat_Content, dtype: int64

In [37]:
# We can do this single line but for illustrative purpose , i kept separatly

data['Item_Fat_Content']=data['Item_Fat_Content'].replace(['Low Fat',"low fat"],'LF')
data['Item_Fat_Content']=data['Item_Fat_Content'].replace(['Regular',"reg"],'Reg')
data['Item_Fat_Content'].value_counts()

LF     4566
Reg    2494
Name: Item_Fat_Content, dtype: int64

In [38]:
#data=data[train.Outlet_Size != "Others"]
y=data["Outlet_Size"]

cp=data.copy(deep=True)

data.drop('Outlet_Size',axis=1,inplace=True)

data=pd.get_dummies(data,drop_first=True)

In [41]:
x_train,x_test,y_train,y_test=train_test_split(data,y,test_size=0.1)

In [42]:
data.head()

dummies=["Item_Fat_Content","Outlet_Identifier","Outlet_Location_Type","Outlet_Type"]

final_train=pd.get_dummies(x_train,drop_first=True)
final_test=pd.get_dummies(x_test,drop_first=True)

In [43]:
final_train.head()


,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,Item_Fat_Content_Reg,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,...,Outlet_Identifier_OUT017,Outlet_Identifier_OUT018,Outlet_Identifier_OUT035,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2
276,16.750,0.081253,256.4988,1987,7452.9652,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1013,14.500,0.063851,260.4594,2004,3139.9128,1,1,0,0,0,...,0,0,1,0,0,0,1,0,1,0
3325,16.200,0.063017,100.3700,2004,998.7000,0,0,0,1,0,...,0,0,1,0,0,0,1,0,1,0
7639,5.615,0.210512,121.3730,1998,123.1730,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1232,19.700,0.161028,254.1014,2002,4845.0266,1,0,0,1,0,...,0,0,0,1,0,0,1,0,1,0


In [44]:
knn_clf=KNeighborsClassifier()

grid_search=GridSearchCV(knn_clf,param_grid,verbose=3) 

grid_search.fit(data,y)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] n_neighbors=3, weights=uniform ..................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...... n_neighbors=3, weights=uniform, score=0.375, total=   0.2s
[CV] n_neighbors=3, weights=uniform ..................................
[CV] ...... n_neighbors=3, weights=uniform, score=0.406, total=   0.1s
[CV] n_neighbors=3, weights=uniform ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s


[CV] ...... n_neighbors=3, weights=uniform, score=0.399, total=   0.1s
[CV] n_neighbors=3, weights=uniform ..................................
[CV] ...... n_neighbors=3, weights=uniform, score=0.391, total=   0.1s
[CV] n_neighbors=3, weights=uniform ..................................
[CV] ...... n_neighbors=3, weights=uniform, score=0.390, total=   0.1s
[CV] n_neighbors=3, weights=distance .................................
[CV] ..... n_neighbors=3, weights=distance, score=0.392, total=   0.1s
[CV] n_neighbors=3, weights=distance .................................
[CV] ..... n_neighbors=3, weights=distance, score=0.416, total=   0.1s
[CV] n_neighbors=3, weights=distance .................................
[CV] ..... n_neighbors=3, weights=distance, score=0.407, total=   0.1s
[CV] n_neighbors=3, weights=distance .................................
[CV] ..... n_neighbors=3, weights=distance, score=0.393, total=   0.1s
[CV] n_neighbors=3, weights=distance .................................
[CV] .

[CV] ...... n_neighbors=9, weights=uniform, score=0.386, total=   0.1s
[CV] n_neighbors=9, weights=uniform ..................................
[CV] ...... n_neighbors=9, weights=uniform, score=0.394, total=   0.1s
[CV] n_neighbors=9, weights=uniform ..................................
[CV] ...... n_neighbors=9, weights=uniform, score=0.387, total=   0.2s
[CV] n_neighbors=9, weights=uniform ..................................
[CV] ...... n_neighbors=9, weights=uniform, score=0.373, total=   0.1s
[CV] n_neighbors=9, weights=uniform ..................................
[CV] ...... n_neighbors=9, weights=uniform, score=0.357, total=   0.1s
[CV] n_neighbors=9, weights=distance .................................
[CV] ..... n_neighbors=9, weights=distance, score=0.382, total=   0.1s
[CV] n_neighbors=9, weights=distance .................................
[CV] ..... n_neighbors=9, weights=distance, score=0.416, total=   0.2s
[CV] n_neighbors=9, weights=distance .................................
[CV] .

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    9.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'n_neighbors': [3, 4, 5, 6, 7, 8, 9, 10],
                          'weights': ['uniform', 'distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [45]:
grid_search.best_params_

{'n_neighbors': 3, 'weights': 'distance'}

In [46]:
grid_search.best_score_

0.4008498583569405

This scoore is pretty low , even if its around 40 we can still use this because , each outlet_identifier has only one Outlet_Size , we are 40 % sure about results , if we take mode out of it we can get value .